In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/13632/Downloads/dataset_stance.csv')

In [3]:
import sklearn

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import nltk

In [7]:
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Error loading averaged_perceptron_tagger: HTTP Error 401:
[nltk_data]     Unauthorized
[nltk_data] Error loading stopwords: HTTP Error 401: Unauthorized


False

In [8]:
pip install "C:/Users/13632/AppData/Roaming/nltk_data/corpora/averaged_perceptron_tagger"

Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


ERROR: Directory 'C:/Users/13632/AppData/Roaming/nltk_data/corpora/averaged_perceptron_tagger' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [9]:
pip install "C:/Users/13632/AppData/Roaming/nltk_data/corpora/stopwords"

Note: you may need to restart the kernel to use updated packages.


ERROR: Directory 'C:/Users/13632/AppData/Roaming/nltk_data/corpora/stopwords' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)'))) - skipping


In [37]:
# WORDNET LEMMATIZER (with appropriate pos tags)


lemmatizer = WordNetLemmatizer()
 
# Define function to lemmatize each word with its POS tag
 
# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
 
def lemmatizing(data):
    lemmatized_data = []
    for i in range(len(data)):
        sentence = data['Tweet'][i]

        # tokenize the sentence and find the POS tag for each token
        word_tokens = nltk.wordpunct_tokenize(sentence) #what does wordpunct_tokenize do?
        pos_tagged = nltk.pos_tag(word_tokens) 

        
        # As you may have noticed, the above pos tags are a little confusing.
        
        # we use our own pos_tagger function to make things simpler to understand.
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
        
        lemmatized_sentence = []
        for word, tag in wordnet_tagged:
            word = word.lower()
            if word.isdigit():
                del word
            elif tag is None:
                # if there is no available tag, append the token as is
                lemmatized_sentence.append(word)
            else:       
                # else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        lemmatized_sentence = " ".join(lemmatized_sentence)
        lemmatized_data.append(lemmatized_sentence)
    return lemmatized_data

In [46]:
#no separation based on subtopic
def grouping(df):
    df_for = []
    df_against = []
    df_neutral = []
    for k in range(len(df)):
        if df['Stance'][k] == 'FAVOR':
            df_for.append(df['Tweet'][k]) 
        elif df['Stance'][k] == 'AGAINST':
            df_against.append(df['Tweet'][k])
        else:
            df_neutral.append(df['Tweet'][k])
    return df_for, df_against, df_neutral

In [11]:
grouped_df = df.groupby(['Target'])
df_atheism = grouped_df.get_group('Atheism')
df_climate = grouped_df.get_group('Climate Change is a Real Concern')
df_trump = grouped_df.get_group('Donald Trump')
df_feminist = grouped_df.get_group('Feminist Movement')
df_clinton = grouped_df.get_group('Hillary Clinton')
df_abortion = grouped_df.get_group('Legalization of Abortion')

consider how to count punctuation too. are they considered by the vectorizer function?

In [31]:
#tfidfVectorizer
def tfidf_vectorizer(list):
    vectorizer = TfidfVectorizer(ngram_range=(1,3))
    matrix = vectorizer.fit_transform(list)
    idf_table = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
    return idf_table

In [35]:
df_atheism = df_atheism.drop('Unnamed: 0', axis='columns')
df_atheism = df_atheism.reset_index()
df_atheism = df_atheism.drop('index', axis = 'columns')

In [38]:
lemmatized_tweets = lemmatizing(df_atheism)

In [45]:
table = tfidf_vectorizer(lemmatized_tweets)

In [40]:
table

,100aeuassembly,100aeuassembly celebrate,100aeuassembly celebrate ethicalunion,100aeuassembly semst,100th,100th anniversary,100th anniversary amazing,10x,10x bad,10x bad bethankful,...,zealot from,zealot from take,zealot lovewins,zealot lovewins semst,zmanoj,zmanoj sanghparivarorg,zmanoj sanghparivarorg when,zubair,zubair ibn,zubair ibn awwam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
table = table.transpose()